<a href="https://colab.research.google.com/github/Sofia-dsc/Repo-Procesamiento-del-habla/blob/main/Datasets_Sof%C3%ADa_Rold%C3%A1n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Actividades

1. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos. Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido la similaridad según el contenido del texto y la etiqueta de clasificación.

2. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación (f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial y ComplementNB.

3. Transponer la matriz documento-término. De esa manera se obtiene una matriz término-documento que puede ser interpretada como una colección de vectorización de palabras. Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente".



# Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [ ]:
# Importo las librerías que voy a necesitar
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

In [ ]:
# Importo el dataset
from sklearn.datasets import fetch_20newsgroups
import numpy as np

In [ ]:
# Separo y cargo los datos en train y test
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

In [ ]:
# Comienzo el vectorizador y accedo al texto
tfidfvect = TfidfVectorizer()
newsgroups_train.data[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [ ]:
# Obtengo el vocabulario y calculo el vector tf-idf
X_train = tfidfvect.fit_transform(newsgroups_train.data)

In [ ]:
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [ ]:
# En y_train guardo los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [ ]:
# Tomo 5 documentos al azar con random
import random

In [ ]:
random.seed(1819)

In [ ]:
docs_azar = random.sample(newsgroups_train.data, 5)

In [ ]:
# Veo que documentos he seleccionado
for i, doc in enumerate(docs_azar):
    print(f"Documento {i + 1}:\n{doc}\n{'-' * 80}\n")

Documento 1:
The floppy is served by DMA on the motherboard,
and original DMA-controller can't reach more than the first
16MB (The address-space of the ISA-bus)
joerg

--------------------------------------------------------------------------------

Documento 2:
I have a Tseng labs video card that gives me problems when I do anything in 
super VGA mode.  CHECKIT v3.0 reports a Video Page Frame Address Error at 
Page Frame #7.  What does this mean and how (if I can) could this be fixed?
The card Says ET4000Ax on it.
--------------------------------------------------------------------------------

Documento 3:

I use the ashtray to keep change and other items in. I converted the 
cigarette lighter into a volume control knob for my in trunk subwoofer!



                                                  .  
                                                 /                
Larry                            __/    _______/_                 
keys@csmes.ncsl.nist.gov       /                  

In [ ]:
## Mido la similaridad de los docs seleccionados y el resto de documentos
for i, doc in enumerate(docs_azar):
    vector = tfidfvect.transform([doc])
    coseno = cosine_similarity(vector, X_train)[0]
    mostsim = np.argsort(coseno)[::-1][1:6]
    print(f'Documento {i} de la clase: {newsgroups_train.target_names[y_train[newsgroups_train.data.index(doc)]]}')

#Visualizo los 5 documentos más similares
    print(f'Los 5 documentos más similares son:')
    for idx in mostsim:
        print(f'Clase: {newsgroups_train.target_names[y_train[idx]]}')


Documento 0 de la clase: comp.sys.ibm.pc.hardware
Los 5 documentos más similares son:
Clase: comp.sys.ibm.pc.hardware
Clase: comp.sys.ibm.pc.hardware
Clase: comp.sys.ibm.pc.hardware
Clase: comp.sys.ibm.pc.hardware
Clase: comp.sys.ibm.pc.hardware
Documento 1 de la clase: comp.sys.ibm.pc.hardware
Los 5 documentos más similares son:
Clase: comp.sys.mac.hardware
Clase: comp.graphics
Clase: comp.sys.mac.hardware
Clase: comp.sys.ibm.pc.hardware
Clase: comp.graphics
Documento 2 de la clase: rec.autos
Los 5 documentos más similares son:
Clase: rec.autos
Clase: rec.autos
Clase: rec.autos
Clase: rec.autos
Clase: comp.windows.x
Documento 3 de la clase: soc.religion.christian
Los 5 documentos más similares son:
Clase: soc.religion.christian
Clase: soc.religion.christian
Clase: soc.religion.christian
Clase: talk.religion.misc
Clase: soc.religion.christian
Documento 4 de la clase: alt.atheism
Los 5 documentos más similares son:
Clase: alt.atheism
Clase: talk.politics.misc
Clase: alt.atheism
Clase: t

In [ ]:
## Entreno los modelos

# Modelo Naive Bayes Multinomial
clf = MultinomialNB()
clf.fit(X_train, y_train)
X_test = tfidfvect.transform(newsgroups_test.data)
y_pred = clf.predict(X_test)
print(f'F1-score (macro): {f1_score(newsgroups_test.target, y_pred, average="macro")}')

F1-score (macro): 0.5854345727938506


In [ ]:
# Modelo Naive Bayes Complement
clf2 = ComplementNB()
clf2.fit(X_train, y_train)
y_pred2 = clf2.predict(X_test)
print(f'F1-score (macro): {f1_score(newsgroups_test.target, y_pred2, average="macro")}')

F1-score (macro): 0.692953349950875


In [ ]:
# Transpongo la matriz documento-término
X_train_transposed = X_train.T

# Pongo las 5 palabras elegidas para evaluar la similaridad
palabras = ['education', 'space', 'graphics', 'business', 'history']

for palabra in palabras:
    idx = tfidfvect.vocabulary_[palabra]
    idx2word = {v: k for k, v in tfidfvect.vocabulary_.items()}
    palabra_vect = X_train_transposed[idx]
    similar_words = cosine_similarity(palabra_vect, X_train_transposed)[0]
    most_similar_idx = np.argsort(similar_words)[::-1][1:6]
    print(f'Palabra elegida: {palabra}')
    for idx in most_similar_idx:
        print(f'Palabra similar encontrada: {idx2word[idx]}')


Palabra elegida: education
Palabra similar encontrada: abstinence
Palabra similar encontrada: pregnancy
Palabra similar encontrada: joplin
Palabra similar encontrada: janis
Palabra similar encontrada: vaunted
Palabra elegida: space
Palabra similar encontrada: nasa
Palabra similar encontrada: seds
Palabra similar encontrada: shuttle
Palabra similar encontrada: enfant
Palabra similar encontrada: seti
Palabra elegida: graphics
Palabra similar encontrada: comp
Palabra similar encontrada: grieggs
Palabra similar encontrada: 3d
Palabra similar encontrada: cfd
Palabra similar encontrada: discused
Palabra elegida: business
Palabra similar encontrada: uwo
Palabra similar encontrada: regina
Palabra similar encontrada: s4lawren
Palabra similar encontrada: gophers
Palabra similar encontrada: reunion
Palabra elegida: history
Palabra similar encontrada: inalcik
Palabra similar encontrada: tibor
Palabra similar encontrada: avigdor
Palabra similar encontrada: halasi
Palabra similar encontrada: kazimir

## **Enfoque, aproximación y experiencia.**

### Utilicé técnicas de vectorización como TF-IDF para transformar textos en representaciones numéricas que puedan ser analizadas, implementé medidas de similitud, como la similitud del coseno, para entender cómo se relacionan los documentos y las palabras. y apliqué métricas como el F1-score (macro) para evaluar el rendimiento de los modelos de clasificación y realizar ajustes en función de los resultados.

### Este trabajo me aportó bastante experiencia nueva porque no conocía este modo de trabajar documentos y buscar similitudes, asique como era nuevo para mi tuve que investigar en internet y basarme en el contenido de las clases.